In [2]:
import sys

In [3]:
sys.path.append("/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/my_engine/chess-sauce")

In [4]:
import my_engine
import chess
import stockfish

In [8]:
sf = stockfish.Stockfish(path="/Users/diegoesparza/Downloads/stockfish/stockfish-macos-x86-64")

In [9]:
sf.set_fen_position("4r3/8/Pk1pN1p1/2p2p2/1p6/1P3BP1/1BK1R2P/8 b - - 3 49")

In [23]:
sf.get_top_moves(20)

[{'Move': 'e2e4', 'Centipawn': 31, 'Mate': None},
 {'Move': 'd2d4', 'Centipawn': 20, 'Mate': None},
 {'Move': 'g1f3', 'Centipawn': 19, 'Mate': None},
 {'Move': 'c2c4', 'Centipawn': 17, 'Mate': None},
 {'Move': 'e2e3', 'Centipawn': 17, 'Mate': None},
 {'Move': 'g2g3', 'Centipawn': 14, 'Mate': None},
 {'Move': 'c2c3', 'Centipawn': 13, 'Mate': None},
 {'Move': 'a2a3', 'Centipawn': 3, 'Mate': None},
 {'Move': 'h2h3', 'Centipawn': -3, 'Mate': None},
 {'Move': 'b1c3', 'Centipawn': -4, 'Mate': None},
 {'Move': 'a2a4', 'Centipawn': -12, 'Mate': None},
 {'Move': 'd2d3', 'Centipawn': -12, 'Mate': None},
 {'Move': 'b2b3', 'Centipawn': -13, 'Mate': None},
 {'Move': 'f2f4', 'Centipawn': -22, 'Mate': None},
 {'Move': 'b2b4', 'Centipawn': -34, 'Mate': None},
 {'Move': 'h2h4', 'Centipawn': -39, 'Mate': None},
 {'Move': 'g1h3', 'Centipawn': -55, 'Mate': None},
 {'Move': 'b1a3', 'Centipawn': -55, 'Mate': None},
 {'Move': 'f2f3', 'Centipawn': -76, 'Mate': None},
 {'Move': 'g2g4', 'Centipawn': -121, 'Mate

In [13]:
sf.will_move_be_a_capture("g6g5")

<Capture.NO_CAPTURE: 'no capture'>

In [32]:
# this is mostly good. one ~flaw is that it could leave the position in a state
# where white has to play a specific move to save a piece from capture. worth
# thinking about whether this would cause the same issue as hanging piece /
# tactic. maybe not because, since white can move the piece out of harms way, it
# won't affect stockfish eval. and we don't expect model to pick up on attacks
# necessarily.
def gen_random_positions(b, sf, positions_to_generate, opening_depth, random_depth):
    fen = b.fen()
    sf.set_fen_position(fen)
    if opening_depth > 0:
        next_opening_depth, next_random_depth = opening_depth-1, random_depth
        moves = sf.get_top_moves(min(positions_to_generate, 7))
        top_centipawn = moves[0]["Centipawn"]
        last_idx = 1
        while True:
            if last_idx == len(moves):
                break
            if last_idx > positions_to_generate:
                break
            if moves[last_idx]["Centipawn"] + 100 < top_centipawn:
                break
            last_idx += 1
            
        moves_to_explore = moves[:last_idx]
            
    else:
        next_opening_depth = 0
        if random_depth == 0:
            next_random_depth = 0
            if positions_to_generate != 1:
                raise ValueError(positions_to_generate)
            moves = sf.get_top_moves(1)
            if sf.will_move_be_a_capture(moves[0]["Move"]) != stockfish.Stockfish.Capture.NO_CAPTURE:
                moves_to_explore = moves
            else:
                print(fen)
                return None
        else:
            next_random_depth = random_depth-1
            # maybe we shouldn't do min, so that branches are spread out across
            # good and bad positions
            moves = sf.get_top_moves(min(positions_to_generate, 20))
            moves_to_explore = moves

    pos_per = positions_to_generate // len(moves_to_explore)
    rem = positions_to_generate - pos_per * len(moves_to_explore)
    for idx, move in enumerate(moves_to_explore):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        move_uci = move["Move"]
        chess_move = chess.Move.from_uci(move_uci)
        b.push(chess_move)
        gen_random_positions(b, sf, next_positions_to_generate, next_opening_depth, next_random_depth)
        b.pop()

In [33]:
b = chess.Board()
positions_to_generate = 5
opening_depth = 20
random_depth = 14
gen_random_positions(b, sf, positions_to_generate, opening_depth, random_depth)

1qrr2k1/1p2ppbp/2npbnp1/p5B1/P3P3/1NN3Q1/1PP1BPPP/3R1RK1 w - - 12 18
r2r2k1/p3npp1/1p2bq1p/2p5/Q2P4/3BPN2/P4PPP/2R2RK1 w - - 0 18
r6r/pp1bbkpp/2n5/5p2/B7/P1N1B3/1P2KPPP/R6R w - - 3 18
r4rk1/ppp2pp1/4bn1p/8/N3PP2/3nB1P1/PP4BP/R4RK1 b - - 0 18
r4rk1/bpq2ppp/p4n2/5b2/NP6/P4B2/3R1PPP/2BQ1RK1 w - - 6 18


In [15]:
print(sf.get_board_visual())

+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b | n | r | 8
+---+---+---+---+---+---+---+---+
| p | p | p | p | p | p | p | p | 7
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
| P | P | P | P | P | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h

